### CLUSTERING TORONTO NEIGHBORHOODS ON THE BASIS OF SIMILAR VENUES


In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
print('libararies imported')

libararies imported


### 1. getting data and and tranforming it to dataframe 

#### using https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M to get list of neighborhoods in toronto


In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_text = requests.get(url).text
print(url)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [3]:
soup = BeautifulSoup(html_text, 'lxml')
table = soup.find('table')
trs = table.find_all('tr')

rows=[]
for tr in trs:
    row=tr.find_all('td')
    if row:
        rows.append(row)

lst=[]
for row in rows:
    postal_code=row[0].text.rstrip()
    borough=row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    
    if borough!='Not assigned':
        if neighborhood=='Not assigned':
            neighborhood = borough
        lst.append([postal_code, borough, neighborhood])
lst

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

In [4]:
df = pd.DataFrame(lst ,columns=['PostalCode','Borough','Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
df.shape

(103, 3)

In [6]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [7]:
#!pip install pgeocode
import pgeocode
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df['PostalCode'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)
    

Postal Code M3A has been retrieved. 102 codes left
Postal Code M4A has been retrieved. 101 codes left
Postal Code M5A has been retrieved. 100 codes left
Postal Code M6A has been retrieved. 99 codes left
Postal Code M7A has been retrieved. 98 codes left
Postal Code M9A has been retrieved. 97 codes left
Postal Code M1B has been retrieved. 96 codes left
Postal Code M3B has been retrieved. 95 codes left
Postal Code M4B has been retrieved. 94 codes left
Postal Code M5B has been retrieved. 93 codes left
Postal Code M6B has been retrieved. 92 codes left
Postal Code M9B has been retrieved. 91 codes left
Postal Code M1C has been retrieved. 90 codes left
Postal Code M3C has been retrieved. 89 codes left
Postal Code M4C has been retrieved. 88 codes left
Postal Code M5C has been retrieved. 87 codes left
Postal Code M6C has been retrieved. 86 codes left
Postal Code M9C has been retrieved. 85 codes left
Postal Code M1E has been retrieved. 84 codes left
Postal Code M4E has been retrieved. 83 codes le

In [8]:
df['Latitude']=latitudes
df['Longitude']=longitudes
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


In [9]:
df.dropna(inplace=True)

### Foursquare Credentials

In [10]:
CLIENT_ID = '5RUVSMYVTJQQCZOXDKXNBJ5R4EAI02SUKLPNDKPCNF4CNPV1' # your Foursquare ID
CLIENT_SECRET = 'JIOE31WTL2J103R5UOSMCXQARPFYLC00M2UFF4LEMJ0FTH15' # your Foursquare Secret
ACCESS_TOKEN = 'GVUNLMHFLCC5P4W0FJXUVDM5ZRGWNO2ZF0PWIQBUCR3CZ0XE'
VERSION = '20210222' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5RUVSMYVTJQQCZOXDKXNBJ5R4EAI02SUKLPNDKPCNF4CNPV1
CLIENT_SECRET:JIOE31WTL2J103R5UOSMCXQARPFYLC00M2UFF4LEMJ0FTH15


### 2. analysing one neighborhood

In [11]:
neigh_name =df.loc[0]['Neighborhood']
neighborhood_latitude= df.loc[0]['Latitude']
neighborhood_longitude= df.loc[0]['Longitude']
print(f'we are going to analys {neigh_name} in this section')

we are going to analys Parkwoods in this section


In [12]:
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URl

'https://api.foursquare.com/v2/venues/explore?&client_id=5RUVSMYVTJQQCZOXDKXNBJ5R4EAI02SUKLPNDKPCNF4CNPV1&client_secret=JIOE31WTL2J103R5UOSMCXQARPFYLC00M2UFF4LEMJ0FTH15&v=20210222&ll=43.7545,-79.33&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '604a8346aa67e67eb874ead6'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7590000045, 'lng': -79.32378161085641},
   'sw': {'lat': 43.7499999955, 'lng': -79.33621838914358}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label'

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
print(venues)    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b', 'name': 'Brookbanks Park', 'location': {'address': 'Toronto', 'lat': 43.751976046055574, 'lng': -79.33214044722958, 'labeledLatLngs': [{'label': 'display', 'lat': 43.751976046055574, 'lng': -79.33214044722958}], 'distance': 329, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}, 'venuePage': {'id': '600917367'}}, 'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInte

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,Yorkmills Wellness & Spa,Spa,43.756800,-79.325346


### 3. writting general function to get vebues and venue categories foe all the neighborhoods

### get nearby venues is function where you pass list of neighboorhoods with there latitudes and longitudes and you get a list of near by venues for all the neighborhoods

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


## 4.analysing the venues and there categories distribution

In [18]:
df_venue = getNearbyVenues(df['Neighborhood'], df['Latitude'],df['Longitude'])
df_venue


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.7545,-79.3300,Yorkmills Wellness & Spa,43.756800,-79.325346,Spa
3,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...
2165,"Mimico NW, The Queensway West, South of Bloor,...",43.6256,-79.5231,Kingsway Boxing Club,43.627254,-79.526684,Gym
2166,"Mimico NW, The Queensway West, South of Bloor,...",43.6256,-79.5231,Solmaz,43.626773,-79.527259,Food & Drink Shop
2167,"Mimico NW, The Queensway West, South of Bloor,...",43.6256,-79.5231,Buon Giorno Cafe,43.622801,-79.519322,Italian Restaurant
2168,"Mimico NW, The Queensway West, South of Bloor,...",43.6256,-79.5231,Queensway Fish & Chips,43.621720,-79.524588,Fish & Chips Shop


In [19]:
print(df_venue.shape)
len(df_venue['Venue Category'].unique())


(2170, 7)


256


#### one hot encoding of the category list

In [20]:
onehot =pd.get_dummies(df_venue['Venue Category'], prefix="", prefix_sep="")

onehot['Neighborhood'] =df_venue['Neighborhood'] 
onehot.set_index('Neighborhood' ,inplace =True)
onehot.reset_index(inplace =True)
onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
onehot.shape

(2170, 256)

In [22]:
grouped = onehot.groupby(onehot['Neighborhood']).mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.142857,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
grouped.shape

(96, 256)

### function to return top 5 venues categories 

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
# code to name the column name according to their sufix
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Badminton Court,Skating Rink,Newsagent,Yoga Studio,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Coffee Shop,Pub,Athletics & Sports,Convenience Store,Dance Studio,Donut Shop,Field
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Fried Chicken Joint,Mediterranean Restaurant,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
3,Bayview Village,Dog Run,Flower Shop,Gas Station,Park,Trail,Escape Room,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Indian Restaurant,Fast Food Restaurant,Butcher,Liquor Store,Café,Restaurant,Sushi Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",Park,Bookstore,Baby Store,Coffee Shop,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
92,Woburn,Korean BBQ Restaurant,Yoga Studio,Flea Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
93,Woodbine Heights,Spa,Beer Store,Convenience Store,Video Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space
94,York Mills West,Convenience Store,Daycare,Park,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [26]:
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# fitting the groupped data 
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 4, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 4,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       3, 0, 0, 2, 1, 0, 2, 0])

In [27]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [28]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Agincourt,Latin American Restaurant,Breakfast Spot,Badminton Court,Skating Rink,Newsagent,Yoga Studio,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
1,0,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Coffee Shop,Pub,Athletics & Sports,Convenience Store,Dance Studio,Donut Shop,Field
2,0,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Fried Chicken Joint,Mediterranean Restaurant,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
3,2,Bayview Village,Dog Run,Flower Shop,Gas Station,Park,Trail,Escape Room,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
4,0,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Indian Restaurant,Fast Food Restaurant,Butcher,Liquor Store,Café,Restaurant,Sushi Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
91,2,"Willowdale, Willowdale West",Park,Bookstore,Baby Store,Coffee Shop,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
92,1,Woburn,Korean BBQ Restaurant,Yoga Studio,Flea Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
93,0,Woodbine Heights,Spa,Beer Store,Convenience Store,Video Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space
94,2,York Mills West,Convenience Store,Daycare,Park,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [29]:
merged = df

In [30]:
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [33]:
merged.dropna(inplace=True)
merged['Cluster Labels'] =merged['Cluster Labels'].astype('int')
merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,2,Spa,Food & Drink Shop,Park,Doner Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4A,North York,Victoria Village,43.7276,-79.3148,0,Hockey Arena,Intersection,Pizza Place,Park,Coffee Shop,Portuguese Restaurant,French Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Gym / Fitness Center,Greek Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Cosmetics Shop,Sandwich Place,Men's Store,Food Court,Jewelry Store,Juice Bar
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Sushi Restaurant,Italian Restaurant,Beer Bar,Burrito Place,Martial Arts School,Café,Escape Room,Mexican Restaurant,Chinese Restaurant,Ramen Restaurant


In [34]:
# create map
map_clusters = folium.Map(location=[43.6532,-1], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
